In this notebook I create a small catalog from the GLADE to the stellar mass estimation.

The code was written by Mária Pálfi (marika97@caesar.elte.hu).

In [1]:
# importing useful packages
import pandas as pd

## Reading the GLADE catalog

Here I used the *z_helio* redshift. *designation* corresponds to *2MASS XSC*.

In [2]:
# reading the catalog to 'data' pandas dataframe
data = pd.read_csv( 'v16_MB_pecvellel', delimiter = ' ',
                   usecols=[2,3,4,5,7,8,9,17,19,20,22,25,26], # necessary columns  
                   header=None, # there are no header in the txt
                   names=[ "GWGC","Hyp", "designation", "wiseX", "type", "ra", "dec", "Kmag",
                          "W1", "W2", "z", "lumdis", 'lumdis_err' ], # I name the columns
                   low_memory = False )
print( 'dataframe with the necessary columns:\n' )
data

dataframe with the necessary columns:



,GWGC,Hyp,designation,wiseX,type,ra,dec,Kmag,W1,W2,z,lumdis,lumdis_err
0,NGC4736,NGC4736,12505314+4107125,NaN,G,192.721451,41.120152,5.169,NaN,NaN,0.000991,4.392418,NaN
1,NGC4548,NGC4548,12352642+1429467,NaN,G,188.860123,14.496320,7.368,NaN,NaN,0.004120,11.201905,NaN
2,NGC6503,NGC6503,17492651+7008396,NaN,G,267.360474,70.144341,7.382,NaN,NaN,0.001000,9.020106,NaN
3,NGC4442,NGC4442,12280389+0948130,NaN,G,187.016220,9.803620,7.381,NaN,NaN,0.003557,9.499651,NaN
4,NGC4469,NGC4469,NaN,NaN,G,187.367000,8.749890,NaN,NaN,NaN,0.004139,10.817553,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23181753,NaN,NaN,NaN,NaN,Q,359.999118,28.954734,NaN,17.728901,16.535843,2.452000,20403.464630,NaN
23181754,NaN,NaN,NaN,NaN,Q,359.999303,34.720842,NaN,16.933657,15.875454,3.109000,27158.368406,NaN
23181755,NaN,NaN,NaN,NaN,Q,359.999615,3.268586,NaN,14.833991,13.702355,1.232962,8792.439985,NaN
23181756,NaN,NaN,NaN,NaN,Q,359.999759,20.721079,NaN,16.979166,16.091722,2.009865,16022.500633,NaN


## Changing the K magnitudes

Cappellari used the *k_m_ext*, but we have *k_m_k20fe*.

https://irsa.ipac.caltech.edu/applications/2MASS/PubGalPS/XSC_key.html

I downloaded the 2MASS data from https://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-dd.

In [3]:
twomass = pd.read_csv( '2mass.csv' )
dic = { 'designation': twomass.designation, 'k_m_ext': twomass.k_m_ext, 
       'k_msig_ext': twomass.k_msig_ext}
twomasswork = pd.DataFrame( dic )

data = pd.merge(data, twomasswork, on=['designation'], how='left')
print( 'merged dataframe:\n' )
data.head()

merged dataframe:



,GWGC,Hyp,designation,wiseX,type,ra,dec,Kmag,W1,W2,z,lumdis,lumdis_err,k_m_ext,k_msig_ext
0,NGC4736,NGC4736,12505314+4107125,NaN,G,192.721451,41.120152,5.169,NaN,NaN,0.000991,4.392418,NaN,5.106,0.016
1,NGC4548,NGC4548,12352642+1429467,NaN,G,188.860123,14.496320,7.368,NaN,NaN,0.004120,11.201905,NaN,7.115,0.031
2,NGC6503,NGC6503,17492651+7008396,NaN,G,267.360474,70.144341,7.382,NaN,NaN,0.001000,9.020106,NaN,7.296,0.021
3,NGC4442,NGC4442,12280389+0948130,NaN,G,187.016220,9.803620,7.381,NaN,NaN,0.003557,9.499651,NaN,7.294,0.024
4,NGC4469,NGC4469,NaN,NaN,G,187.367000,8.749890,NaN,NaN,NaN,0.004139,10.817553,NaN,NaN,NaN


## Adding the WISE W1 magnitude errors 

from wiseScosSvm Table (http://ssa.roe.ac.uk/www/ssa_browser.html)

Check *wise_mag_err.ipynb*.

In [4]:
w1err_frame = pd.read_csv( 'wise_w1err.txt', delimiter = '\t' )
w1err_frame['wiseX'] = [ i.strip() for i in w1err_frame.wiseX.values ]
data = pd.merge( w1err_frame, data, on=['wiseX'], how='right' )
print( 'merged dataframe:\n' )
data.head()

merged dataframe:



,wiseX,w1mCorr,w1mCorrErr,fromAllSky,GWGC,Hyp,designation,type,ra,dec,Kmag,W1,W2,z,lumdis,lumdis_err,k_m_ext,k_msig_ext
0,NaN,NaN,NaN,NaN,NGC4736,NGC4736,12505314+4107125,G,192.721451,41.120152,5.169,NaN,NaN,0.000991,4.392418,NaN,5.106,0.016
1,NaN,NaN,NaN,NaN,NGC4548,NGC4548,12352642+1429467,G,188.860123,14.496320,7.368,NaN,NaN,0.004120,11.201905,NaN,7.115,0.031
2,NaN,NaN,NaN,NaN,NGC6503,NGC6503,17492651+7008396,G,267.360474,70.144341,7.382,NaN,NaN,0.001000,9.020106,NaN,7.296,0.021
3,NaN,NaN,NaN,NaN,NGC4442,NGC4442,12280389+0948130,G,187.016220,9.803620,7.381,NaN,NaN,0.003557,9.499651,NaN,7.294,0.024
4,NaN,NaN,NaN,NaN,NGC4469,NGC4469,NaN,G,187.367000,8.749890,NaN,NaN,NaN,0.004139,10.817553,NaN,NaN,NaN


## Writing out the dataframe

In [5]:
data.to_csv( 'data_for_work.txt', sep = '\t', index = False, header = True )